In [1]:
import sys
import os

sys.path.append(os.path.abspath("../src"))

import generations
import search_space
import data_preprocessing as dp


In [2]:
DATA_ID = 54
X_train, y_train, X_val, y_val, X_test, y_test = dp.get_preprocessed_data(
        dataset_id=DATA_ID, scaling=True, random_seed=13, return_as='tensor', task_type='classification')

# sample row 56
print(X_train[56], y_train[56])

Loading Dataset: vehicle
Class column is not numeric. Applying LabelEncoder.
Data loaded successfully! Format: tensor
Training data shape: torch.Size([540, 18])
y_training data shape: torch.Size([540])
tensor([ 0.4196,  0.0118,  0.3634,  0.1528,  0.8222,  0.4100, -0.3638,  0.2742,
        -0.2267,  0.6259, -0.5015, -0.4306, -0.5676, -0.7777,  0.3033, -0.0679,
         0.5025,  0.5874]) tensor(3)


In [3]:
input_size, output_size = dp.get_tensor_sizes(X_train, y_train)
search_space = search_space.SearchSpace(input_size=input_size, output_size=output_size)

In [13]:
N_INDIVIDUALS = 100 # amount of architectures to be evaluated as a starting point
N_EPOCHS = 100
percentile_drop = 15 # drop the worst 5% of architectures after first epoch

In [14]:
generation = generations.Generation(search_space, N_INDIVIDUALS, starting_instances=100, seed=13)

In [20]:
df = generation.build_snapshot().copy()
display(df.head())

,id,batch_size,n_instances,epochs_trained,efforts,arch_hidden_layers,arch_activation_fn,arch_dropout_rate,arch_optimizer_type,arch_learning_rate,...,test_loss,test_acc,forecasted_val_acc,score,forecast_gain,fcst_greater_than_baseline,last_train_acc,last_train_loss,last_val_acc,last_val_loss
11,107,32,"[100, 200, 400, 800, 1080, 1080]",5,"[0.18518518518518517, 0.7407407407407407, 2.22...","[145, 382, 479, 186]",<class 'torch.nn.modules.activation.GELU'>,0.0,<class 'torch.optim.adamw.AdamW'>,0.006258,...,[],[],0.732876,0.425503,-0.017124,False,0.807407,0.393177,0.720588,0.497395
20,122,32,"[100, 200, 400, 800, 1080]",4,"[0.18518518518518517, 0.7407407407407407, 2.22...","[145, 382, 479, 186]",<class 'torch.nn.modules.activation.GELU'>,0.0,<class 'torch.optim.adamw.AdamW'>,0.006258,...,[],[],0.728622,0.432625,-0.021378,False,0.753704,0.520096,0.735294,0.546057
29,167,64,"[100, 200, 400, 800]",3,"[0.18518518518518517, 0.7407407407407407, 2.22...","[33, 431]",<class 'torch.nn.modules.activation.Tanh'>,0.1,<class 'torch.optim.adamw.AdamW'>,0.014338,...,[],[],0.694129,0.387946,-0.055871,False,0.607500,0.946556,0.683824,0.680054
15,111,64,"[100, 200, 400, 800, 1080, 1080]",5,"[0.18518518518518517, 0.7407407407407407, 2.22...","[411, 431]",<class 'torch.nn.modules.activation.ELU'>,0.4,<class 'torch.optim.adamw.AdamW'>,0.002022,...,[],[],0.681088,0.373906,-0.068912,False,0.859259,0.359902,0.669118,0.611996
23,136,64,"[100, 200, 400, 800, 1080]",4,"[0.18518518518518517, 0.7407407407407407, 2.22...","[33, 431]",<class 'torch.nn.modules.activation.Tanh'>,0.1,<class 'torch.optim.adamw.AdamW'>,0.014338,...,[],[],0.673740,0.388614,-0.076260,False,0.718519,0.689726,0.698529,0.809434


In [16]:
generation.run_ebe(epochs=N_EPOCHS,
                    X_train=X_train,
                    y_train=y_train,
                    X_val=X_val,
                    y_val=y_val, 
                    percentile_drop=percentile_drop,
                    baseline_metric=0.75, 
                    time_budget=15,
                    epoch_threshold=3, 
                    track_all_models=True
                    )

Built models: 100
Epoch 1/100


c:\Users\user\OneDrive - Universidad de la Sabana\Documents\GitHub\ShouldYouEvenNN\src\generations.py:377: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pd.concat([self.cumulative_snapshot, self.current_snapshot])


Built models: 100
Epoch 2/100
Built models: 115
Epoch 3/100
Built models: 140
Epoch 4/100
Built models: 175
Epoch 5/100
Built models: 220
Epoch 6/100
Built models: 270
Time budget exceeded at epoch 7: 16.30 seconds
EBE process completed.


,id,batch_size,n_instances,epochs_trained,efforts,arch_hidden_layers,arch_activation_fn,arch_dropout_rate,arch_optimizer_type,arch_learning_rate,...,test_loss,test_acc,forecasted_val_acc,score,forecast_gain,fcst_greater_than_baseline,last_train_acc,last_train_loss,last_val_acc,last_val_loss
13,107,32,"[100, 200, 400, 800, 1080, 1080]",5,"[0.18518518518518517, 0.7407407407407407, 2.22...","[145, 382, 479, 186]",<class 'torch.nn.modules.activation.GELU'>,0.0,<class 'torch.optim.adamw.AdamW'>,0.006258,...,[],[],0.732876,0.425503,-0.017124,False,0.807407,0.393177,0.720588,0.497395
22,122,32,"[100, 200, 400, 800, 1080]",4,"[0.18518518518518517, 0.7407407407407407, 2.22...","[145, 382, 479, 186]",<class 'torch.nn.modules.activation.GELU'>,0.0,<class 'torch.optim.adamw.AdamW'>,0.006258,...,[],[],0.728622,0.432625,-0.021378,False,0.753704,0.520096,0.735294,0.546057
31,167,64,"[100, 200, 400, 800]",3,"[0.18518518518518517, 0.7407407407407407, 2.22...","[33, 431]",<class 'torch.nn.modules.activation.Tanh'>,0.1,<class 'torch.optim.adamw.AdamW'>,0.014338,...,[],[],0.694129,0.387946,-0.055871,False,0.607500,0.946556,0.683824,0.680054
17,111,64,"[100, 200, 400, 800, 1080, 1080]",5,"[0.18518518518518517, 0.7407407407407407, 2.22...","[411, 431]",<class 'torch.nn.modules.activation.ELU'>,0.4,<class 'torch.optim.adamw.AdamW'>,0.002022,...,[],[],0.681088,0.373906,-0.068912,False,0.859259,0.359902,0.669118,0.611996
25,136,64,"[100, 200, 400, 800, 1080]",4,"[0.18518518518518517, 0.7407407407407407, 2.22...","[33, 431]",<class 'torch.nn.modules.activation.Tanh'>,0.1,<class 'torch.optim.adamw.AdamW'>,0.014338,...,[],[],0.673740,0.388614,-0.076260,False,0.718519,0.689726,0.698529,0.809434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,266,64,"[100, 200]",1,[0.18518518518518517],"[33, 334, 236]",<class 'torch.nn.modules.activation.Tanh'>,0.1,<class 'torch.optim.adamw.AdamW'>,0.014338,...,[],[],0.000000,0.070588,-0.750000,False,0.340000,1.636433,0.617647,1.128359
96,267,1024,"[100, 200]",1,[0.18518518518518517],"[145, 34, 440, 293, 28]",<class 'torch.nn.modules.activation.GELU'>,0.3,<class 'torch.optim.adamw.AdamW'>,0.000328,...,[],[],0.000000,0.000000,-0.750000,False,0.230000,2.011346,0.257353,1.461142
97,268,1024,"[100, 200]",1,[0.18518518518518517],"[33, 34, 440, 293, 28]",<class 'torch.nn.modules.activation.Tanh'>,0.3,<class 'torch.optim.adamw.AdamW'>,0.000328,...,[],[],0.000000,0.000000,-0.750000,False,0.260000,1.553890,0.205882,1.467177
98,269,64,"[100, 200]",1,[0.18518518518518517],"[33, 431]",<class 'torch.nn.modules.activation.ELU'>,0.1,<class 'torch.optim.adamw.AdamW'>,0.002022,...,[],[],0.000000,0.000000,-0.750000,False,0.260000,1.559236,0.477941,1.235544


In [17]:
generation.cumulative_snapshot

,id,batch_size,n_instances,epochs_trained,efforts,arch_hidden_layers,arch_activation_fn,arch_dropout_rate,arch_optimizer_type,arch_learning_rate,...,test_loss,test_acc,forecasted_val_acc,score,forecast_gain,fcst_greater_than_baseline,last_train_acc,last_train_loss,last_val_acc,last_val_loss
0,0,512,"[100, 200]",1,[0.18518518518518517],"[379, 212, 131, 257, 204, 392, 153, 239, 292, ...",<class 'torch.nn.modules.activation.ELU'>,0.5,<class 'torch.optim.sgd.SGD'>,0.079400,...,[],[],0.0,0.000000,-0.75,False,0.21,2.198337,0.235294,1.390514
1,1,128,"[100, 200]",1,[0.18518518518518517],"[147, 455, 401, 445, 327, 334, 435, 321, 96, 3...",<class 'torch.nn.modules.activation.ReLU'>,0.2,<class 'torch.optim.adamw.AdamW'>,0.017149,...,[],[],0.0,0.000000,-0.75,False,0.27,1.388567,0.235294,46.764997
2,2,32,"[100, 200]",1,[0.18518518518518517],"[453, 330, 217, 273, 66, 475, 240, 365, 312, 8...",<class 'torch.nn.modules.activation.ReLU'>,0.0,<class 'torch.optim.adamw.AdamW'>,0.000451,...,[],[],0.0,0.000000,-0.75,False,0.20,1.389405,0.330882,1.383757
3,3,256,"[100, 200]",1,[0.18518518518518517],"[132, 15, 112, 365, 5, 400, 339, 356, 479, 16,...",<class 'torch.nn.modules.activation.ReLU'>,0.3,<class 'torch.optim.rmsprop.RMSprop'>,0.000633,...,[],[],0.0,0.000000,-0.75,False,0.23,17.497604,0.227941,1.408948
4,4,256,"[100, 200]",1,[0.18518518518518517],"[30, 363, 243, 471, 205, 459, 82, 497, 208, 94...",<class 'torch.nn.modules.activation.Tanh'>,0.3,<class 'torch.optim.adam.Adam'>,0.037352,...,[],[],0.0,0.000000,-0.75,False,0.21,1.466536,0.272059,1.827237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,266,64,"[100, 200]",1,[0.18518518518518517],"[33, 334, 236]",<class 'torch.nn.modules.activation.Tanh'>,0.1,<class 'torch.optim.adamw.AdamW'>,0.014338,...,[],[],0.0,0.070588,-0.75,False,0.34,1.636433,0.617647,1.128359
96,267,1024,"[100, 200]",1,[0.18518518518518517],"[145, 34, 440, 293, 28]",<class 'torch.nn.modules.activation.GELU'>,0.3,<class 'torch.optim.adamw.AdamW'>,0.000328,...,[],[],0.0,0.000000,-0.75,False,0.23,2.011346,0.257353,1.461142
97,268,1024,"[100, 200]",1,[0.18518518518518517],"[33, 34, 440, 293, 28]",<class 'torch.nn.modules.activation.Tanh'>,0.3,<class 'torch.optim.adamw.AdamW'>,0.000328,...,[],[],0.0,0.000000,-0.75,False,0.26,1.553890,0.205882,1.467177
98,269,64,"[100, 200]",1,[0.18518518518518517],"[33, 431]",<class 'torch.nn.modules.activation.ELU'>,0.1,<class 'torch.optim.adamw.AdamW'>,0.002022,...,[],[],0.0,0.000000,-0.75,False,0.26,1.559236,0.477941,1.235544


In [9]:
df = generation.starting_snapshot
display(df)

,id,batch_size,n_instances,epochs_trained,efforts,arch_hidden_layers,arch_activation_fn,arch_dropout_rate,arch_optimizer_type,arch_learning_rate,...,test_loss,test_acc,forecasted_val_acc,score,forecast_gain,fcst_greater_than_baseline,last_train_acc,last_train_loss,last_val_acc,last_val_loss
0,0,512,"[100, 200]",0,[0.18518518518518517],"[379, 212, 131, 257, 204, 392, 153, 239, 292, ...",<class 'torch.nn.modules.activation.ELU'>,0.5,<class 'torch.optim.sgd.SGD'>,0.079400,...,[],[],0.0,0.0,0.0,False,None,None,None,None
1,1,128,"[100, 200]",0,[0.18518518518518517],"[147, 455, 401, 445, 327, 334, 435, 321, 96, 3...",<class 'torch.nn.modules.activation.ReLU'>,0.2,<class 'torch.optim.adamw.AdamW'>,0.017149,...,[],[],0.0,0.0,0.0,False,None,None,None,None
2,2,32,"[100, 200]",0,[0.18518518518518517],"[453, 330, 217, 273, 66, 475, 240, 365, 312, 8...",<class 'torch.nn.modules.activation.ReLU'>,0.0,<class 'torch.optim.adamw.AdamW'>,0.000451,...,[],[],0.0,0.0,0.0,False,None,None,None,None
3,3,256,"[100, 200]",0,[0.18518518518518517],"[132, 15, 112, 365, 5, 400, 339, 356, 479, 16,...",<class 'torch.nn.modules.activation.ReLU'>,0.3,<class 'torch.optim.rmsprop.RMSprop'>,0.000633,...,[],[],0.0,0.0,0.0,False,None,None,None,None
4,4,256,"[100, 200]",0,[0.18518518518518517],"[30, 363, 243, 471, 205, 459, 82, 497, 208, 94...",<class 'torch.nn.modules.activation.Tanh'>,0.3,<class 'torch.optim.adam.Adam'>,0.037352,...,[],[],0.0,0.0,0.0,False,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,1024,"[100, 200, 400, 800, 1080, 1080]",0,"[0.18518518518518517, 0.7407407407407407, 2.22...","[214, 421, 236, 77, 354, 361, 392, 108, 52, 41...",<class 'torch.nn.modules.activation.LeakyReLU'>,0.0,<class 'torch.optim.adamw.AdamW'>,0.020008,...,[],[],0.0,0.0,0.0,False,None,None,None,None
96,96,64,"[100, 200, 400, 800, 1080]",0,"[0.18518518518518517, 0.7407407407407407, 2.22...","[332, 489, 254, 123, 397, 356, 199, 187, 344, ...",<class 'torch.nn.modules.activation.ELU'>,0.4,<class 'torch.optim.adam.Adam'>,0.000581,...,[],[],0.0,0.0,0.0,False,None,None,None,None
97,97,1024,"[100, 200, 400, 800, 1080]",0,"[0.18518518518518517, 0.7407407407407407, 2.22...","[160, 30, 12, 122, 110, 159]",<class 'torch.nn.modules.activation.Sigmoid'>,0.2,<class 'torch.optim.rmsprop.RMSprop'>,0.015682,...,[],[],0.0,0.0,0.0,False,None,None,None,None
98,98,256,"[100, 200, 400, 800, 1080]",0,"[0.18518518518518517, 0.7407407407407407, 2.22...","[363, 232, 69, 422, 465, 421, 152, 452, 100]",<class 'torch.nn.modules.activation.ELU'>,0.5,<class 'torch.optim.rmsprop.RMSprop'>,0.013242,...,[],[],0.0,0.0,0.0,False,None,None,None,None


In [10]:
generation.cumulative_snapshot['id'].max()

np.int64(270)

In [18]:
generation.individuals_created

270

In [19]:
generation.starting_snapshot

,id,batch_size,n_instances,epochs_trained,efforts,arch_hidden_layers,arch_activation_fn,arch_dropout_rate,arch_optimizer_type,arch_learning_rate,...,test_loss,test_acc,forecasted_val_acc,score,forecast_gain,fcst_greater_than_baseline,last_train_acc,last_train_loss,last_val_acc,last_val_loss
0,0,512,"[100, 200]",0,[0.18518518518518517],"[379, 212, 131, 257, 204, 392, 153, 239, 292, ...",<class 'torch.nn.modules.activation.ELU'>,0.5,<class 'torch.optim.sgd.SGD'>,0.079400,...,[],[],0.0,0.0,0.0,False,None,None,None,None
1,1,128,"[100, 200]",0,[0.18518518518518517],"[147, 455, 401, 445, 327, 334, 435, 321, 96, 3...",<class 'torch.nn.modules.activation.ReLU'>,0.2,<class 'torch.optim.adamw.AdamW'>,0.017149,...,[],[],0.0,0.0,0.0,False,None,None,None,None
2,2,32,"[100, 200]",0,[0.18518518518518517],"[453, 330, 217, 273, 66, 475, 240, 365, 312, 8...",<class 'torch.nn.modules.activation.ReLU'>,0.0,<class 'torch.optim.adamw.AdamW'>,0.000451,...,[],[],0.0,0.0,0.0,False,None,None,None,None
3,3,256,"[100, 200]",0,[0.18518518518518517],"[132, 15, 112, 365, 5, 400, 339, 356, 479, 16,...",<class 'torch.nn.modules.activation.ReLU'>,0.3,<class 'torch.optim.rmsprop.RMSprop'>,0.000633,...,[],[],0.0,0.0,0.0,False,None,None,None,None
4,4,256,"[100, 200]",0,[0.18518518518518517],"[30, 363, 243, 471, 205, 459, 82, 497, 208, 94...",<class 'torch.nn.modules.activation.Tanh'>,0.3,<class 'torch.optim.adam.Adam'>,0.037352,...,[],[],0.0,0.0,0.0,False,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,1024,"[100, 200, 400, 800, 1080, 1080]",0,"[0.18518518518518517, 0.7407407407407407, 2.22...","[214, 421, 236, 77, 354, 361, 392, 108, 52, 41...",<class 'torch.nn.modules.activation.LeakyReLU'>,0.0,<class 'torch.optim.adamw.AdamW'>,0.020008,...,[],[],0.0,0.0,0.0,False,None,None,None,None
96,96,64,"[100, 200, 400, 800, 1080]",0,"[0.18518518518518517, 0.7407407407407407, 2.22...","[332, 489, 254, 123, 397, 356, 199, 187, 344, ...",<class 'torch.nn.modules.activation.ELU'>,0.4,<class 'torch.optim.adam.Adam'>,0.000581,...,[],[],0.0,0.0,0.0,False,None,None,None,None
97,97,1024,"[100, 200, 400, 800, 1080]",0,"[0.18518518518518517, 0.7407407407407407, 2.22...","[160, 30, 12, 122, 110, 159]",<class 'torch.nn.modules.activation.Sigmoid'>,0.2,<class 'torch.optim.rmsprop.RMSprop'>,0.015682,...,[],[],0.0,0.0,0.0,False,None,None,None,None
98,98,256,"[100, 200, 400, 800, 1080]",0,"[0.18518518518518517, 0.7407407407407407, 2.22...","[363, 232, 69, 422, 465, 421, 152, 452, 100]",<class 'torch.nn.modules.activation.ELU'>,0.5,<class 'torch.optim.rmsprop.RMSprop'>,0.013242,...,[],[],0.0,0.0,0.0,False,None,None,None,None


I don't know why does the starting snapshot gets updated or something